In [31]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
import wikipediaapi
import google.generativeai as genai
import PyPDF2
from langchain_community.document_loaders import WebBaseLoader
import requests
from bs4 import BeautifulSoup
from langchain.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma

In [32]:
# loader = WebBaseLoader("https://en.wikipedia.org/wiki/Artificial_intelligence")
# loader.requests_kwargs = {'verify':False}
# data = loader.load()

wiki_wiki = wikipediaapi.Wikipedia(
    user_agent='Giskard application',
    language='en',
    extract_format=wikipediaapi.ExtractFormat.WIKI
)
p_wiki = wiki_wiki.page('Virat_Kohli')
data = p_wiki.text

In [33]:
# data = str(data)
# type(data)
# len(data)

In [34]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000 , chunk_overlap = 10)
docs = text_splitter.create_documents([data])

In [35]:
len(docs)

47

In [47]:
#local models

# llm = Ollama(model = "llama3")
embeddings = OllamaEmbeddings(model= "nomic-embed-text")

In [37]:
#local vectordb

# db = Chroma.from_documents(docs , embeddings , persist_directory="./chroma_db")

In [38]:
import google.generativeai as genai
import dotenv

api_key = dotenv.get_key(".env", key_to_get = "GEMINI_API_KEY")
genai.configure(api_key=api_key)

# gen_config = genai.GenerationConfig(
#     temperature = 0.1
# )
model = genai.GenerativeModel("models/gemini-1.5-flash")

In [39]:
for m in genai.list_models():
    if "embedContent" in m.supported_generation_methods:
        print(m)

Model(name='models/embedding-001',
      base_model_id='',
      version='001',
      display_name='Embedding 001',
      description='Obtain a distributed representation of a text.',
      input_token_limit=2048,
      output_token_limit=1,
      supported_generation_methods=['embedContent'],
      temperature=None,
      top_p=None,
      top_k=None)
Model(name='models/text-embedding-004',
      base_model_id='',
      version='004',
      display_name='Text Embedding 004',
      description='Obtain a distributed representation of a text.',
      input_token_limit=2048,
      output_token_limit=1,
      supported_generation_methods=['embedContent'],
      temperature=None,
      top_p=None,
      top_k=None)


In [44]:
# response = model.generate_content("embedding model for gemini")
# response.text

"I can't provide you with the exact embedding model used by Gemini. This is because:\n\n* **It's proprietary:** Google keeps the details of its large language model architectures and specific embedding models confidential. This is common for companies developing advanced AI systems, as it protects their intellectual property and competitive advantage.\n* **It's constantly evolving:** Gemini is likely under continuous development, with improvements and changes being made to its architecture and embedding models.\n\nHowever, I can give you some general insights into the type of embedding models used in large language models like Gemini:\n\n**What are Embeddings?**\n\n* Embeddings are a way to represent words, phrases, or even entire documents as numerical vectors. These vectors capture the meaning and relationship of these elements in a way that can be understood by a machine learning model.\n* In the context of language models, embeddings are crucial for tasks like:\n    * **Understandi

In [41]:
# from langchain_google_genai import GoogleGenerativeAIEmbeddings

# embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=api_key)

# vector = embeddings.embed_query("hello, world!")
# vector[:5]

In [50]:
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(docs, embeddings)

In [51]:
db.save_local("faiss_index")

In [54]:
retriever = db.as_retriever()

In [55]:
query = "who is the wife of virat kohli"
context = retriever.invoke(query)
context

[Document(page_content="Other honours and awards\nPeople's Choice Awards India for Favourite Sportsperson: 2012\nGQ Sportsman of the year: 2013\nCNN-News18 Indian of the Year: 2017\nPeople for the Ethical Treatment of Animals (PETA) India's Person of the Year: 2019\nDelhi & District Cricket Association (DDCA) renamed a stand after Kohli at Feroz Shah Kotla, Delhi.\n\nSee also\nSports in India – Overview of sport in India\nList of players who have scored 10,000 or more runs in One Day International cricket\nList of cricketers by number of international centuries scored\nList of cricketers who have scored centuries in both innings of a Test match\nList of cricketers with centuries in all international formats\nList of most-followed Instagram accounts\n\nNotes\nReferences\nBibliography\nExternal links\n\nOfficial website\nVirat Kohli at ESPNcricinfo\nVirat Kohli on Instagram\nVirat Kohli on X\nVirat Kohli at IMDb"),
 Document(page_content='Outside cricket\nPersonal life\nKohli\'s romantic